In [1]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile
import LibEE

import pandas as pd
import numpy as np


In [2]:
#for sfs
# !pip install mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

def Selection(x, y, forward):        
    # 기존 학습
    model = RandomForestRegressor(n_estimators=1000, min_samples_leaf=1)

    # model = XGBRegressor(max_delta_step=0.0, objective = 'reg:squarederror', nthread=-1,
    #                      n_estimators = 100,
    #                      colsample_bytree=0.9168,
    #                      learning_rate=0.1284,
    #                      max_depth=7,
    #                      min_child_weight=5,
    #                      subsample=0.4541)

    sfs = SequentialFeatureSelector(model,
                                    k_features= 20, 
                                    forward=forward, # if forward = True then SFS otherwise SBS
                                    floating=True, 
                                    verbose=2,
                                    scoring='neg_mean_absolute_error',
                                    n_jobs=2,
                                    cv=5
                                    )

    #after applying sfs fit the data:
    # DataFrame 객체는 안됨. 넘파이객체만 됨
    sfs.fit(x.to_numpy(), y['pv_claim'].to_numpy())

    return sfs

In [3]:
x = pd.read_csv('x.csv')
y = pd.read_csv('y.csv')


In [4]:
sffs = Selection(x, y, True)
print(sffs.k_feature_names_)
LibUtils.ELoop()

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
def GetSelectedFeatures(x, sfs, colCnt):
    cols = pd.DataFrame({ 'idx': range(len(x.columns)), 'colName': x.columns })
    selectedColIdxs = pd.DataFrame(sfs.subsets_[colCnt]['feature_idx'], columns=['idx'])
    selectedColIdxs = pd.merge(selectedColIdxs, cols, on='idx', how='left')
    del selectedColIdxs['idx']
    return selectedColIdxs['colName'].to_numpy()

for i in range(10, 16+1):
    res = GetSelectedFeatures(x, sffs, i)
    print(res)

['Sex' 'Prem' 'PremYr' 'AccumPrem' 'StartAge' 'FundAllo02' 'IR14'
 'Moneyness' 'AttainedAge' 'ee_ProdCode_02']
['Sex' 'Prem' 'PremYr' 'AccumPrem' 'ContAmt' 'StartAge' 'FundAllo02'
 'IR14' 'Moneyness' 'AttainedAge' 'ee_ProdCode_02']
['ElapsedMth' 'Sex' 'Prem' 'PremYr' 'AccumPrem' 'ContAmt' 'StartAge'
 'FundAllo02' 'IR14' 'Moneyness' 'AttainedAge' 'ee_ProdCode_02']
['Sex' 'Prem' 'PremYr' 'AccumPrem' 'ContAmt' 'StartAge' 'FundAllo02'
 'IR06' 'IR12' 'IR14' 'Moneyness' 'AttainedAge' 'ee_ProdCode_02']
['Sex' 'Prem' 'PremYr' 'AccumPrem' 'ContAmt' 'StartAge' 'FundAllo02'
 'IR06' 'IR09' 'IR12' 'IR14' 'Moneyness' 'AttainedAge' 'ee_ProdCode_02']
['ElapsedMth' 'Sex' 'Prem' 'PremYr' 'AccumPrem' 'ContAmt' 'StartAge'
 'FundAllo02' 'IR06' 'IR09' 'IR12' 'IR14' 'Moneyness' 'AttainedAge'
 'ee_ProdCode_02']
['ElapsedMth' 'Sex' 'Prem' 'PremYr' 'AccumPrem' 'ContAmt' 'StartAge'
 'FundAllo01' 'FundAllo02' 'IR06' 'IR09' 'IR12' 'IR14' 'Moneyness'
 'AttainedAge' 'ee_ProdCode_02']


In [ ]:
sffs.score